# Machine Learning Engineer Nanodegree

## Model Evaluation & Validation

## Project 1: Predicting Boston Housing Prices

Welcome to the first project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and you will need to implement additional functionality to successfully complete this project. You will not need to modify the included code beyond what is requested. Sections that begin with 'Implementation' in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a 'TODO' statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a 'Question X' header. Carefully read each question and provide thorough answers in the following text boxes that begin with 'Answer:'. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.

## Getting Started

In this project, you will evaluate the performance and predictive power of a model that has been trained and tested on data collected from homes in suburbs of Boston, Massachusetts. A model trained on this data that is seen as a good fit could then be used to make certain predictions about a home — in particular, its monetary value. This model would prove to be invaluable for someone like a real estate agent who could make use of such information on a daily basis.

The dataset for this project originates from the UCI Machine Learning Repository. The Boston housing data was collected in 1978 and each of the 506 entries represent aggregated data about 14 features for homes from various suburbs in Boston, Massachusetts. For the purposes of this project, the following preprocessing steps have been made to the dataset:

* 16 data points have an 'MEDV' value of 50.0. These data points likely contain missing or censored values and have been removed.

* 1 data point has an 'RM' value of 8.78. This data point can be considered an outlier and has been removed.

* The features 'RM', 'LSTAT', 'PTRATIO', and 'MEDV' are essential. The remaining non-relevant features have been excluded

* The feature 'MEDV' has been multiplicatively scaled to account for 35 years of market inflation.

Run the code cell below to load the Boston housing dataset, along with a few of the necessary Python libraries required for this project. You will know the dataset loaded successfully if the size of the dataset is reported.

In [ ]:
# Import libraries necessary for this project
import numpy as np
import pandas as pd
from sklearn.cross_validation import ShuffleSplit

import visuals as vs

# Load the Boston housing dataset
data = pd.read_csv('housing.csv')
prices = data['MEDV']
features = data.drop('MEDV', axis = 1)

## Data Exploration

In this first section of this project, you will make a cursory investigation about the Boston housing data and provide your observations. Familiarizing yourself with the data through an explorative process is a fundamental practice to help you better understand and justify your results.

Since the main goal of this project is to construct a working model which has the capability of predicting the value of houses, we will need to separate the dataset into features and the target variable. The features, 'RM', 'LSTAT', and 'PTRATIO', give us quantitative information about each data point. The target variable, 'MEDV', will be the variable we seek to predict. These are stored in features and prices, respectively.

### Implementation: Calculate Statistics

For your very first coding implementation, you will calculate descriptive statistics about the Boston housing prices. Since numpy has already been imported for you, use this library to perform the necessary calculations. These statistics will be extremely important later on to analyze various prediction results from the constructed model.

In the code cell below, you will need to implement the following:

* Calculate the minimum, maximum, mean, median, and standard deviation of 'MEDV', which is stored in prices.
 * Store each calculation in their respective variable.

In [ ]:
# TODO: Minimum price of the data
minimum_price = np.min(prices)

# TODO: Maximum price of the data
maximum_price = np.max(prices)

# TODO: Mean price of the data
mean_price = np.mean(prices)

# TODO: Median price of the data
median_price = np.median(prices)

# TODO: Standard deviation of prices of the data
std_price = np.std(prices)

# Show the calculated statistics
print("\nStatistics for Boston housing dataset:\n")
print("Minimum price: ${:,.2f}".format(minimum_price))
print("Maximum price: ${:,.2f}".format(maximum_price))
print("Mean price: ${:,.2f}".format(mean_price))
print("Median price ${:,.2f}".format(median_price))
print("Standard deviation of prices: ${:,.2f}".format(std_price))

Statistics for Boston housing dataset:

Minimum price: \$105,000.00

Maximum price: \$1,024,800.00

Mean price: \$454,342.94

Median price \$438,900.00

Standard deviation of prices: \$165,171.13


## Question 1 - Feature Observation

As a reminder, we are using three features from the Boston housing dataset: 'RM', 'LSTAT', and 'PTRATIO'. For each data point (neighborhood):

* 'RM' is the average number of rooms among homes in the neighborhood.
* 'LSTAT' is the percentage of homeowners in the neighborhood considered "lower class" (working poor).
* 'PTRATIO' is the ratio of students to teachers in primary and secondary schools in the neighborhood.

### Answer:

Let's examine things carefully:
        
    Since these features reflect the quality and level of life in these neighborhoods,
    it is natural to assume that possible increases or decreases in value will determine
    the demand of the market over the houses, which will reflect in higher or lower
    prices. 
        
    An increase of the values of LSTAT feature, which represents the percentage 
    of lower class people living in a neighborhood, won't be approaching a 
    "better neighborhood" value, and as a result an increase of this feature more 
    likely will decrease the value of the house prices, since wealthy people, who 
    are willing to pay more, want to live in wealthier areas. 
      
    An increase in RM i.e. if the average number of rooms per house increase, 
    which means if the size of the houses is getting bigger, will imply that the 
    neighborhood is wealthier, since large houses are found in wealthier areas. 
    That means, that the prices will also go up, as the size of the houses is 
    going up. 
    
    For the PTRATIO, if its values increase, it means that there are many students
    per teacher, which means that on average the level of education is decreased. 
    Prestigious schools are famous for having small size classes so that the students
    will get more individual attention by the teacher and thus comprehend the
    material better. Non prestigious schools, like most public schools, are famous 
    (or infamous) for the opposite: large size classrooms, prioritizing quantity 
    over quality. Thus, increased PTRATIO values means the level of education 
    is decreasing, which means that wealthy potential buyers won't be much interested
    in such neighborhoods cause they want their kids to go to good schools, 
    which in turn will decrease the demand, and as a result the prices of the 
    houses in these neighborhoods. 

    In summary:
        
        As RM increases -> MEDV increases
        As LSTAT increases -> MEDV decreases
        As PTRATION increases -> MEDV decreases
        
## Developing a Model

In this second section of the project, you will develop the tools and techniques necessary for a model to make a prediction. Being able to make accurate evaluations of each model's performance through the use of these tools and techniques helps to greatly reinforce the confidence in your predictions.

### Implementation: Define a Performance Metric

It is difficult to measure the quality of a given model without quantifying its performance over training and testing. This is typically done using some type of performance metric, whether it is through calculating some type of error, the goodness of fit, or some other useful measurement. For this project, you will be calculating the coefficient of determination, R2, to quantify your model's performance. The coefficient of determination for a model is a useful statistic in regression analysis, as it often describes how "good" that model is at making predictions.

The values for R2 range from 0 to 1, which captures the percentage of squared correlation between the predicted and actual values of the target variable. A model with an R2 of 0 always fails to predict the target variable, whereas a model with an R2 of 1 perfectly predicts the target variable. Any value between 0 and 1 indicates what percentage of the target variable, using this model, can be explained by the features. A model can be given a negative R2 as well, which indicates that the model is no better than one that naively predicts the mean of the target variable.

For the performance_metric function in the code cell below, you will need to implement the following:

* Use r2_score from sklearn.metrics to perform a performance calculation between y_true and y_predict.
* Assign the performance score to the score variable.

In [ ]:
# TODO: Import 'r2_score'
from sklearn.metrics import r2_score

def performance_metric(y_true, y_predict):
    """ Calculates and returns the performance score between 
        true and predicted values based on the metric chosen. """
    
    # TODO: Calculate the performance score between 'y_true' and 'y_predict'
    score = r2_score(y_true, y_predict)
    
    # Return the score
    return score

# Calculate the performance of this model
score = performance_metric([3, -0.5, 2, 7, 4.2], [2.5, 0.0, 2.1, 7.8, 5.3])
print("\nModel has a coefficient of determination, R^2, of {:.3f}.".format(score))

## Question 2 - Goodness of Fit

Assume that a dataset contains five data points and a model made the following predictions for the target variable:

Would you consider this model to have successfully captured the variation of the target variable? Why or why not?

Run the code cell below to use the performance_metric function and calculate this model's coefficient of determination. 


In [ ]:
# Calculate the performance of this model
score = performance_metric([3, -0.5, 2, 7, 4.2], [2.5, 0.0, 2.1, 7.8, 5.3])
print "Model has a coefficient of determination, R^2, of {:.3f}.".format(score)

Answer:

 Yes. The output of the above metric given the data above is 0.923, which is in a  
    neighborhood of 1 with radius = 0.077. Sounds pretty good score given that
    a score of 1 perfectly predicts the target variable. 
    
### Implementation: Shuffle and Split Data

Your next implementation requires that you take the Boston housing dataset and split the data into training and testing subsets. Typically, the data is also shuffled into a random order when creating the training and testing subsets to remove any bias in the ordering of the dataset.

For the code cell below, you will need to implement the following:

* Use train_test_split from sklearn.cross_validation to shuffle and split the features and prices data into training and testing sets.

    * Split the data into 80% training and 20% testing.
    * Set the random_state for train_test_split to a value of your choice. This ensures results are consistent.
    
* Assign the train and testing splits to X_train, X_test, y_train, and y_test.

In [ ]:
# TODO: Import 'train_test_split'
from sklearn.cross_validation import train_test_split

# TODO: Shuffle and split the data into training and testing subsets
X_train, X_test, y_train, y_test = train_test_split(features, prices, test_size=20, random_state=True)

# Success
print("Training and testing split was successful.")

## Question 3 - Training and Testing

What is the benefit to splitting a dataset into some ratio of training and testing subsets for a learning algorithm?

Answer:

    It is useful to split the data into training and testing subsets because that
    way we can use some of them (most of them usually) to train the model, and then 
    use the rest of the data to test the model. That enables us to extract information
    about the model, how well it performs, if it overfits or underfits the data etc. 
    
    If we don't split the data and use all of them for training, then we won't 
    have any way of knowing how the model might perform in real data. This is a
    good technique to help us evaluate our model choices. 
    
    (This is from lesson 6: testing models)
    

## Analyzing Model Performance

In this third section of the project, you'll take a look at several models' learning and testing performances on various subsets of training data. Additionally, you'll investigate one particular algorithm with an increasing 'max_depth' parameter on the full training set to observe how model complexity affects performance. Graphing your model's performance based on varying criteria can be beneficial in the analysis process, such as visualizing behavior that may not have been apparent from the results alone.

### Learning Curves

The following code cell produces four graphs for a decision tree model with different maximum depths. Each graph visualizes the learning curves of the model for both training and testing as the size of the training set is increased. Note that the shaded region of a learning curve denotes the uncertainty of that curve (measured as the standard deviation). The model is scored on both the training and testing sets using R2, the coefficient of determination.

Run the code cell below and use these graphs to answer the following question.

In [ ]:
# Produce learning curves for varying training set sizes and maximum depths
vs.ModelLearning(features, prices)

### Question 4 - Learning the Data

Choose one of the graphs above and state the maximum depth for the model. What happens to the score of the training curve as more training points are added? What about the testing curve? Would having more training points benefit the model?

Answer:

I am choosing the second curve with max_depth = 3, which is highly biased. Deeper-depth
trees give better accuracy on the training set, and less deep-depth trees give
better accuracy on the testing set. Moreover, deep trees let the model overfit
the data and create a closely fit decision boundary which often does not
correspond to the actual boundary.
    
With:
   max_depth: 3
   Testing score: blue curve
   Training score: red curve
    
We have the following conclusions:
   * The score of training curve decreases the more we add training points and at some point stabilizes -- it's still high
   * The score of testing curve increases the more we add training points and at some point stabilizes -- it's still high
          
The more training points we have, the lesser the training score will be and the 
higher the testing score will be until they converge, which implies a highly 
biased model.

### Complexity Curves

The following code cell produces a graph for a decision tree model that has been trained and validated on the training data using different maximum depths. The graph produces two complexity curves — one for training and one for validation. Similar to the learning curves, the shaded regions of both the complexity curves denote the uncertainty in those curves, and the model is scored on both the training and validation sets using the performance_metric function.

Run the code cell below and use this graph to answer the following two questions.

In [ ]:
vs.ModelComplexity(X_train, y_train)

### Question 5 - Bias-Variance Tradeoff

When the model is trained with a maximum depth of 1, does the model suffer from high bias or from high variance? How about when the model is trained with a maximum depth of 10? What visual cues in the graph justify your conclusions?

Answer:

 When the model is trained with a max depth of 1, then it suffers from high bias
    When the model is trained with a max depth of 10, then it suffers from high variance.
    The above statements are obvious from the curves, by observing how they behave
    as # of depth -> inf. 
    The more the # of depth grows, the training score is pretty high, while the 
    validation score decreases. That means that as the # of depth increases, the more
    our model "memorizes" the data instead of learning from them, which makes the validation
    curve to decrease, cause the model won't validate as good with high bias.
    
### Question 6 - Best-Guess Optimal Model

Which maximum depth do you think results in a model that best generalizes to unseen data? What intuition lead you to this answer?

Answer: 

Ideally we would want our model to have a good trade off between variance and bias.
That is, we don't want our model to neither overfit (high variance) nor underfit
(high bias). To achieve that, I believe that the max_depth should be 3 or 4.
    
I arrived to this conclusion by looking at the graphs. The first graph with the 
4 plots shows us that a good fit and trade off is achieved with depth 3 versus
the other graphs with increased depth. Then the second graph with the validation 
curve shows that the validation score is in an acceptable region between max depth
3 and 4.  


## Evaluating Model Performance

In this final section of the project, you will construct a model and make a prediction on the client's feature set using an optimized model from fit_model.

### Question 7 - Grid Search

What is the grid search technique and how it can be applied to optimize a learning algorithm?

Answer:

The grid search technique is used when we have a set of models to choose from, 
which differ from each other in their parameter values that lie on a gird.
We then train each model and evaluate them using cross-validation and select 
the one that performed best. 
    
Further research in wikipedia (https://en.wikipedia.org/wiki/Hyperparameter_(machine_learning)):
        
Hyperparameter optimization is the problem of choosing a set of optimal 
hyperparameters for a learning algorithm.
        
The traditional way of performing hyperparameter optimization is grid search,
which is simply an exhaustive searching through a manually specified subset 
of the hyperparameter space of a learning algorithm. 
       
A grid search algorithm must be guided by some performance metric, 
typically measured by cross-validation on the training set or evaluation 
on a held-out validation set.


### Question 8 - Cross-Validation

What is the k-fold cross-validation training technique? What benefit does this technique provide for grid search when optimizing a model?

Answer:

K-fold validation is when we split our data into K subsets and use each subset
separately as testing data, and the rest points as training data. We then train 
our model K different times, one for each subset, and then we average the results
to get the final model. 
    
To do grid search, iterate through a wide range of grid combinations. 
Then, choose the set of parameters for which k-folds reports the lowest error. 


## Implementation: Fitting a Model

Your final implementation requires that you bring everything together and train a model using the decision tree algorithm. To ensure that you are producing an optimized model, you will train the model using the grid search technique to optimize the 'max_depth' parameter for the decision tree. The 'max_depth' parameter can be thought of as how many questions the decision tree algorithm is allowed to ask about the data before making a prediction. Decision trees are part of a class of algorithms called supervised learning algorithms.

For the fit_model function in the code cell below, you will need to implement the following:

* Use DecisionTreeRegressor from sklearn.tree to create a decision tree regressor object.
  * Assign this object to the 'regressor' variable.
  
* Create a dictionary for 'max_depth' with the values from 1 to 10, and assign this to the 'params' variable.

* Use make_scorer from sklearn.metrics to create a scoring function object.
  * Pass the performance_metric function as a parameter to the object.
  * Assign this scoring function to the 'scoring_fnc' variable.

* Use GridSearchCV from sklearn.grid_search to create a grid search object.
  * Pass the variables 'regressor', 'params', 'scoring_fnc', and 'cv_sets' as parameters to the object.
  * Assign the GridSearchCV object to the 'grid' variable.


In [ ]:
# TODO: Import 'make_scorer', 'DecisionTreeRegressor', and 'GridSearchCV'
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import make_scorer
from sklearn.grid_search import GridSearchCV

def fit_model(X, y):
    """ Performs grid search over the 'max_depth' parameter for a 
        decision tree regressor trained on the input data [X, y]. """
    
    # Create cross-validation sets from the training data
    # sklearn version 0.18: ShuffleSplit(n_splits=10, test_size=0.1, train_size=None, random_state=None)
    # sklearn versiin 0.17: ShuffleSplit(n, n_iter=10, test_size=0.1, train_size=None, random_state=None)
    cv_sets = ShuffleSplit(X.shape[0], n_iter = 10, test_size = 0.20, random_state = 0)
  
    # TODO: Create a decision tree regressor object
    regressor = DecisionTreeRegressor(random_state = 0)

    # TODO: Create a dictionary for the parameter 'max_depth' with a range from 1 to 10
    params = {"max_depth" : [1,2,3,4,5,6,7,8,9,10]}

    # TODO: Transform 'performance_metric' into a scoring function using 'make_scorer' 
    scoring_fnc = make_scorer(performance_metric)

    # TODO: Create the grid search cv object --> GridSearchCV()
    # Make sure to include the right parameters in the object:
    # (estimator, param_grid, scoring, cv) which have values 'regressor', 'params', 'scoring_fnc', and 'cv_sets' respectively.
    grid = GridSearchCV(regressor, params, scoring_fnc, cv=cv_sets)

    # Fit the grid search object to the data to compute the optimal model
    grid = grid.fit(X, y)

    # Return the optimal model after fitting the data
    return grid.best_estimator_

# Fit the training data to the model using grid search
reg = fit_model(X_train, y_train)

# Produce the value for 'max_depth'
print("Parameter 'max_depth' is {} for the optimal model.".format(reg.get_params()['max_depth']))

### Making Predictions

Once a model has been trained on a given set of data, it can now be used to make predictions on new sets of input data. In the case of a decision tree regressor, the model has learned what the best questions to ask about the input data are, and can respond with a prediction for the target variable. You can use these predictions to gain information about data where the value of the target variable is unknown — such as data the model was not trained on.


### Question 9 - Optimal Model

What maximum depth does the optimal model have? How does this result compare to your guess in Question 6?

In [ ]:
# Fit the training data to the model using grid search
reg = fit_model(X_train, y_train)

# Produce the value for 'max_depth'
print "Parameter 'max_depth' is {} for the optimal model.".format(reg.get_params()['max_depth'])

Parameter 'max_depth' is 4 for the optimal model.

Answer:

"Parameter 'max_depth' is 4 for the optimal model."
    
In Question 6, by looking at the plots I assumed a max depth of 3 or 4. 
Apparently there is another correlation here that I missed. If you look at 
the second graph, the validation reaches a maximum value at depth 4. If this is
true, then our job becomes easier to quickly get an idea of how the max depth would
look like for our model.   


### Question 10 - Predicting Selling Prices

Imagine that you were a real estate agent in the Boston area looking to use this model to help price homes owned by your clients that they wish to sell. You have collected the following information from three of your clients:

What price would you recommend each client sell his/her home at? Do these prices seem reasonable given the values for the respective features?

In [ ]:
# Produce a matrix for client data
client_data = [[5, 17, 15], # Client 1
               [4, 32, 22], # Client 2
               [8, 3, 12]]  # Client 3

# Show predictions
for i, price in enumerate(reg.predict(client_data)):
    print ("Predicted selling price for Client {}'s home: ${:,.2f}".format(i+1, price))

Predicted selling price for Client 1's home: \$409,400.00

Predicted selling price for Client 2's home: \$229,682.35

Predicted selling price for Client 3's home: \$938,053.85


Answer:
  
Client 1 has a 5-bedroom house (which is large enough), a decent poverty level
at 17%, and a good student/teacher ratio. We can't draw any conclusions by solely
examining this client. So let's look at the other two.
    
Client 2 has a 4-room house (which is relatively smaller), higher poverty rate, 
almost double in fact, and a border-line average student/teacher ratio. Without
looking at the predicted price, We should expect it to be lower since all the 
characteristics are worse than Client 1's characteristics. 
Indeed the price makes sense to be smaller at $231,675
    
Client 3 has a significantly larger house (8 rooms is a palace!), almost no lower
class population (3% poverty), and a great student/teacher ratio, much better
than both of the other clients. Thus, we expect the price to be much higher. 
Indeed the predicted price is $931,350, which is more than double from Client 1.
I would personally expect it to be around double, but I guess the poverty rate 
which itself is almost 6 times less than Client 1's neighborhood makes the difference, 
along with the extra 3 bedrooms. 


### Sensitivity

An optimal model is not necessarily a robust model. Sometimes, a model is either too complex or too simple to sufficiently generalize to new data. Sometimes, a model could use a learning algorithm that is not appropriate for the structure of the data given. Other times, the data itself could be too noisy or contain too few samples to allow a model to adequately capture the target variable — i.e., the model is underfitted. Run the code cell below to run the fit_model function ten times with different training and testing sets to see how the prediction for a specific client changes with the data it's trained on.

In [ ]:
vs.PredictTrials(features, prices, fit_model, client_data)

### Question 11 - Applicability

In a few sentences, discuss whether the constructed model should or should not be used in a real-world setting.

Answer:

Data collected in 1978 are actually pretty old. Many things happened ever since, 
the inflation increased by 276.6%, city projects, perhaps extreme natural conditions
(snowstorms, floods, etc.) that might have changed the preferences of the people.
Nevertheless, one can argue that the prices in the housing market remained analogous
to those at the time. It could be true that the prices only increased percentage-wise
thus not being affected much by all these events, but only their value to have changed. 
    
    
I think the data are reflecting the house prices, however there might be some
valuable input ignored. For example, the age of the property, its condition,
if there is an open space etc matter a lot in the price. Although we want fewer
and as most descriptive factors as possible, a couple more factors would be great. 
On the other hand, the current factors are still pretty descriptive if we admit that
there is a positive correlation between lower class population and higher student/teacher
ratio and criminality. That makes a "criminality rate" factor redundant, for instance. 
    
The model's output is quite reasonable, although not very robust. In particular, 
compared to the max price value we computed above, it has approximately 8% of it. 
Ideally I would like it to be around 3%-4% or less. 
    
Depending on the size of the neighborhood, it is reasonable to estimate a range
in which the price of an individual house should be in. We can then classify (cluster)
house prices per basic neighborhood characteristics, but then it would be very bold
to predict the price of an individual house, there it's likely this house to be an 
exception (outlier).

In general, if I was to sell such software to a real estate agent I would ask for
more recent data -- data from the previous 5 year up to today. Then I will try to 
work a bit more on the rebustness of it, and perhaps compute more metrics to make sure
all is good. Finally I would test it for a month or two, and then deliver it.   